Remove MTL in downstream, i.e. STL with patient data.

Remove attention in pretraining, and remove transformer inputs, use `ablation_noattn_notransformer` folder for pretrained

In [ ]:
import pandas as pd
import numpy as np
import pickle
import torch
import torch.nn as nn
import math

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import yaml
import pprint
import os
import wandb
import sys
import random
from scipy.stats import mode, pearsonr
import pickle
import itertools
import sys
sys.path.append("../")

from src.gaussian_multinomial_diffusion import GaussianMultinomialDiffusion
from src.modules import MLPDiffusion
from src.vae_model import vae
from src.loss_functions import get_kld_loss, coral
from model_definition import *

In [ ]:
fold = 2

In [ ]:
# global variables

CONFIG_PATH = f"../experiment_settings_yaml/ablation/model_config_2A_annotated_mutations_v7_fold{fold}_noattn_notransformer.yaml" # model config path
pretty_print = pprint.PrettyPrinter()
print(f"Loading config from {CONFIG_PATH}")
config = yaml.safe_load(open(CONFIG_PATH))
model_config = config["model_hyperparams"]
folder_config = config["folder_config"]
wandb_config = config["wandb_config"]
wandb_config["project_name"] = wandb_config["project_name"] + f"-{model_config['experiment_id']}-{model_config['experiment_settings']}-fold{model_config['sample_id']}" # updates wandb project name for ease of monitoring and logging.
device = torch.device(f"cuda:{model_config['device']}" if torch.cuda.is_available() else "cpu")
genes_324 = list(pd.read_csv(f"{folder_config['data_folder']}/raw/metadata/gene2ind.txt", header=None)[0])
drug_fp = pd.read_csv(f"{folder_config['data_folder']}/raw/metadata/drug_morgan_fingerprints.csv", index_col=0)
suffixes = ["_piu_max", "_piu_sum", "_piu_mean", "_piu_count",
            "_lu_max", "_lu_sum", "_lu_mean", "_lu_count",
            "_ncu_max", "_ncu_sum", "_ncu_mean", '_ncu_count',
            "_pathogenic_max", "_pathogenic_sum", "_pathogenic_mean", "_pathogenic_count",
            "_vus_max", "_vus_sum", "_vus_mean", "_vus_count",
            "_benign_max", "_benign_sum", "_benign_mean", "_benign_count"
           ]
genes_7776 = []
for s in suffixes:
    for g in list(pd.read_csv(f"{folder_config['data_folder']}/raw/metadata/gene2ind.txt", header=None)[0]):
        genes_7776.append(f"{g}{s}")

# setting up wandb
os.environ["WANDB_CACHE_DIR"] = wandb_config["wandb_cache_dir"]
os.environ["WANDB_DIR"] = wandb_config["wandb_cache_dir"]
wandb.login(key=wandb_config["api_key"])

# seeding
torch.manual_seed(model_config["seed"])
random.seed(model_config["seed"])
np.random.seed(model_config["seed"])
# reproducibility in data loading - https://pytorch.org/docs/stable/notes/randomness.html
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(model_config["seed"])



In [ ]:
pretrained_folder = "/data/ajayago/papers_data/DiffDRP_v7/run_files/saved_model_annotated_mutations/ablation/ablation_noattn_notransformer/"

In [ ]:
# pass samples through the VAE and DDPM network, till just before VAE decoder
def vae_decoder_input(df, vae, diff_model):
    """
    Takes input df, pretrained vae and diffusion model as inputs, runs forward pass till VAE decoder
    """
    with torch.no_grad():
        batch = torch.tensor(df.values) # convert to torch tensor
        inp_vae = batch.to(device, dtype=torch.float32)
        inp, mu, logvar, _ = vae(inp_vae) # From VAE encoder + reparameterization
        
        noise = torch.randn_like(inp) # this is the label we use   
        b = inp.shape[0]
        t = (torch.ones((b,)) * 700).long().to(device) # fixing time steps to 700
        pt = torch.ones_like(t).float() / diff_model.num_timesteps
        inp_t = diff_model.gaussian_q_sample(inp, t, noise) # forward process with cell line model encoder
        
        model_out = diff_model._denoise_fn(inp_t, t) # predicted epsilon from patient decoder
    
        # predict inp from noise using patient model
        inp_pred = diff_model._predict_xstart_from_eps(inp_t, t, model_out)

    return inp_pred.detach().cpu().numpy()

def load_pretrained_models():
    pretrained = torch.load(f"{pretrained_folder}/best_pretrained_validation_loss_{model_config['model_save_criteria']}_{model_config['experiment_id']}_{model_config['experiment_settings']}_fold{model_config['sample_id']}.pth")
    is_real = True if model_config["input_data_type"] == "binary_mutations" else False
    # patients
    patient_vae = vae(input_dim=model_config["feature_num"], k_list=model_config["patient_vae_k_list"], actf_list=model_config["patient_vae_actf_list"], is_real=is_real).to(device)
    tcga_mlp_diffusion_model = MLPDiffusion(d_in=model_config["patient_vae_k_list"][-1]//2, num_classes=0, is_y_cond=False, rtdl_params={"d_layers": [model_config["patient_vae_k_list"][-1]//4], "dropout": model_config["dropout"]}).to(device)
    tcga_diff_model = GaussianMultinomialDiffusion(num_classes=np.array([0]), num_numerical_features=model_config["patient_vae_k_list"][-1]//2, denoise_fn=tcga_mlp_diffusion_model, device=device)#.to(device)
    tcga_diff_model.load_state_dict(pretrained["patient_diff_model"])
    patient_vae.load_state_dict(pretrained["patient_vae_conditioned"])
    # cell lines
    cl_vae = vae(input_dim=model_config["feature_num"], k_list=model_config["cl_vae_k_list"], actf_list=model_config["cl_vae_actf_list"], is_real=is_real).to(device)
    cl_mlp_diffusion_model = MLPDiffusion(d_in=model_config["cl_vae_k_list"][-1]//2, num_classes=0, is_y_cond=False, rtdl_params={"d_layers": [model_config["cl_vae_k_list"][-1]//4], "dropout": model_config["dropout"]}).to(device)
    cl_diff_model = GaussianMultinomialDiffusion(num_classes=np.array([0]), num_numerical_features=model_config["cl_vae_k_list"][-1]//2, denoise_fn=cl_mlp_diffusion_model, device=device)#.to(device)
    cl_diff_model.load_state_dict(pretrained["cl_diff_model"])
    cl_vae.load_state_dict(pretrained["cl_vae_conditioned"])
    return cl_diff_model, cl_vae, tcga_diff_model, patient_vae

def load_datasets(sample_id):
    """
    Takes sample_id as input, loads source and target train, validation and test splits (predefined files from Processing folder).
    """
    data_dir = folder_config["data_folder"] + "input_types/"
    # navigate based on input type
    if model_config["input_data_type"] == "binary_mutations":
        data_dir = data_dir + "raw_mutations/"
        features2select = genes_324  # inclusive of Morgan drug fingerprints of 2048 dim
    elif model_config["input_data_type"] == "annotated_mutations":
        data_dir = data_dir + "annotated_mutations/"
        features2select = genes_7776  # inclusive of Morgan drug fingerprints of 2048 dim
    elif model_config["input_data_type"] == "transformer_inputs": # processed by PREDICT-AI transformer embedder
        data_dir = data_dir + "transformer_inputs_transformed_797/"
        features2select = [f"transformer_embedded_{i}" for i in range(797)] # after transformer embedding
    else:
        print("Unsupported input type!")
        return
    
    # navigate based on experiment id
    if model_config["experiment_id"] == "1A":
        data_dir = data_dir + "Experiment1/SettingA/"
    elif model_config["experiment_id"] == "1B":
        data_dir = data_dir + "Experiment1/SettingB/"
    elif model_config["experiment_id"] == "2A":
        data_dir = data_dir + "Experiment2/SettingA/"
    elif model_config["experiment_id"] == "2B":
        data_dir = data_dir + "Experiment2/SettingB/"
    else:
        print("Unsupported experiment ID!")
        return
    
    # load the fold based on sample_id - Note: cell lines have only 1 fold (fold 0)
    with open(f"{data_dir}/cell_lines_fold0_processed.pkl", "rb") as f:
        source_data = pickle.load(f)

    with open(f"{data_dir}/patients_fold{sample_id}_processed.pkl", "rb") as f:
        target_data = pickle.load(f)

    # load pretrained TCGA VAE and diffusion models
    # pass data points through patient DDPM and get the input to VAE decoder for DRP
    cl_diff_model, cl_vae, tcga_diff_model, patient_vae = load_pretrained_models()
    
    # select data based on experiment settings 
    # Can be CISPLATIN, PACLITAXEL, FLUOROURACIL, SORAFENIB for 1A, CISPLATIN, TCGA-CESC; CISPLATIN, TCGA-HNSC; PACLITAXEL, TCGA-BRCA; FLUOROURACIL, TCGA-STAD for 1B
    # ALL for 2A, TCGA-BRCA, TCGA-CESC, TCGA-HNSC, TCGA-STAD for 2B
    if model_config["experiment_id"] in ["1A", "2B"]:
        setting = model_config["experiment_settings"]
        train_source_data, val_source_data, test_source_data = source_data["train"][setting], source_data["val"][setting], source_data["test"][setting]
        train_target_data, val_target_data, test_target_data = target_data["train"][setting], target_data["val"][setting], target_data["test"][setting]
    elif model_config["experiment_id"] == "1B":
        setting = (model_config["experiment_settings"].split(", ")[0], model_config["experiment_settings"].split(", ")[1], "TCGA")
        train_source_data, val_source_data, test_source_data = source_data["train"][setting], source_data["val"][setting], source_data["test"][setting]
        train_target_data, val_target_data, test_target_data = target_data["train"][setting], target_data["val"][setting], target_data["test"][setting]
    elif model_config["experiment_id"] == "2A":
        train_source_data, val_source_data, test_source_data = source_data["train"], source_data["val"], source_data["test"]
        train_target_data, val_target_data, test_target_data = target_data["train"], target_data["val"], target_data["test"]
    else:
        print("Unsupported experiment settings and ID")
        return
    
    # merge dataframes with drug Morgan fingprint dataframes
    train_source_data_merged = train_source_data.merge(drug_fp, left_on="drug_name", right_on=drug_fp.index)
    val_source_data_merged = val_source_data.merge(drug_fp, left_on="drug_name", right_on=drug_fp.index)
    test_source_data_merged = test_source_data.merge(drug_fp, left_on="drug_name", right_on=drug_fp.index)

    train_target_data_merged = train_target_data.merge(drug_fp, left_on="drug_name", right_on=drug_fp.index)
    val_target_data_merged = val_target_data.merge(drug_fp, left_on="drug_name", right_on=drug_fp.index)
    test_target_data_merged = test_target_data.merge(drug_fp, left_on="drug_name", right_on=drug_fp.index)

    assert train_source_data_merged.shape[0] == train_source_data.shape[0], "Train source data loss after merge!"
    assert val_source_data_merged.shape[0] == val_source_data.shape[0], "Val source data loss after merge!"
    assert test_source_data_merged.shape[0] == test_source_data.shape[0], "Test source data loss after merge!"
    assert train_target_data_merged.shape[0] == train_target_data.shape[0], "Train target data loss after merge!"
    assert val_target_data_merged.shape[0] == val_target_data.shape[0], "Val target data loss after merge!"
    assert test_target_data_merged.shape[0] == test_target_data.shape[0], "Test target data loss after merge!"

    # separate out into input, drug and labels
    train_source_inputs, val_source_inputs, test_source_inputs = train_source_data_merged[features2select], val_source_data_merged[features2select], test_source_data_merged[features2select]
    # pass cl samples through cl diff model and vae
    train_source_inputs_vae = pd.DataFrame(vae_decoder_input(train_source_inputs, cl_vae, cl_diff_model), columns=[f"vae_feat{i}" for i in range(model_config["cl_vae_k_list"][-1]//2)], index=train_source_data_merged.index)
    val_source_inputs_vae = pd.DataFrame(vae_decoder_input(val_source_inputs, cl_vae, cl_diff_model), columns=[f"vae_feat{i}" for i in range(model_config["cl_vae_k_list"][-1]//2)], index=val_source_data_merged.index)
    test_source_inputs_vae = pd.DataFrame(vae_decoder_input(test_source_inputs, cl_vae, cl_diff_model), columns=[f"vae_feat{i}" for i in range(model_config["cl_vae_k_list"][-1]//2)], index=test_source_data_merged.index)
    train_source_drugs, val_source_drugs, test_source_drugs = train_source_data_merged[[str(i) for i in range(0, 2048)]].values, val_source_data_merged[[str(i) for i in range(0, 2048)]].values, test_source_data_merged[[str(i) for i in range(0, 2048)]].values
    train_source_labels, val_source_labels, test_source_labels = train_source_data_merged["auc"].values, val_source_data_merged["auc"].values, test_source_data_merged["auc"].values

    train_target_inputs, val_target_inputs, test_target_inputs = train_target_data_merged[features2select], val_target_data_merged[features2select], test_target_data_merged[features2select]
    # pass patient samples through tcga diff model and vae
    train_target_inputs_vae = pd.DataFrame(vae_decoder_input(train_target_inputs, patient_vae, tcga_diff_model), columns=[f"vae_feat{i}" for i in range(model_config["patient_vae_k_list"][-1]//2)], index=train_target_data_merged.sample_id)
    val_target_inputs_vae = pd.DataFrame(vae_decoder_input(val_target_inputs, patient_vae, tcga_diff_model), columns=[f"vae_feat{i}" for i in range(model_config["patient_vae_k_list"][-1]//2)], index=val_target_data_merged.sample_id)
    test_target_inputs_vae = pd.DataFrame(vae_decoder_input(test_target_inputs, patient_vae, tcga_diff_model), columns=[f"vae_feat{i}" for i in range(model_config["patient_vae_k_list"][-1]//2)], index=test_target_data_merged.sample_id)

    train_target_drugs, val_target_drugs, test_target_drugs = train_target_data_merged[[str(i) for i in range(0, 2048)]].values, val_target_data_merged[[str(i) for i in range(0, 2048)]].values, test_target_data_merged[[str(i) for i in range(0, 2048)]].values
    train_target_labels, val_target_labels, test_target_labels = train_target_data_merged["recist"].values, val_target_data_merged["recist"].values, test_target_data_merged["recist"].values

    return train_source_inputs_vae, train_source_drugs, train_source_labels, val_source_inputs_vae, val_source_drugs, val_source_labels, test_source_inputs_vae, test_source_drugs, test_source_labels, train_target_inputs_vae, train_target_drugs, train_target_labels, val_target_inputs_vae, val_target_drugs, val_target_labels, test_target_inputs_vae, test_target_drugs, test_target_labels, train_target_data_merged, val_target_data_merged, test_target_data_merged, train_source_data_merged, val_source_data_merged, test_source_data_merged

    # pass # needs to return (train_source_data, train_source_labels, val_source_data, val_source_labels, test_source_data, test_source_labels), (train_target_data, train_target_labels, val_target_data, val_target_labels, test_target_data, test_target_labels)
    #  Dummy data
    # train_source_data, val_source_data, test_source_data = np.random.rand(32, 2048 + 4), np.random.rand(10, 2048 + 4), np.random.rand(5, 2048 + 4)
    # train_source_labels, val_source_labels, test_source_labels = np.random.randint(2, size=32), np.random.randint(2, size=10), np.random.randint(2, size=5)
    # train_target_data, val_target_data, test_target_data = np.random.rand(32, 2048 + 4), np.random.rand(10, 2048 + 4), np.random.rand(3, 2048 + 4)
    # train_target_labels, val_target_labels, test_target_labels = np.random.randint(2, size=32), np.random.randint(2, size=10), np.random.randint(2, size=3)
    # return train_source_data, train_source_labels, val_source_data, val_source_labels, test_source_data, test_source_labels, train_target_data, train_target_labels, val_target_data, val_target_labels, test_target_data, test_target_labels

def load_augmented_cl_dataset(sample_id):
    augmented_cl_df = pd.read_csv(f"{pretrained_folder}/augmented_cl_clconditioned_uda_v2_vaeinput_{model_config['model_save_criteria']}_{model_config['experiment_id']}_{model_config['experiment_settings']}_fold{model_config['sample_id']}.csv", index_col=0)
    print(f"Loaded augmented CL data: {augmented_cl_df.shape}")
    return augmented_cl_df

In [ ]:
cl_diff_model, cl_vae, tcga_diff_model, patient_vae = load_pretrained_models()

In [ ]:
train_source_inputs_vae, train_source_drugs, train_source_labels, val_source_inputs_vae, val_source_drugs, val_source_labels, test_source_inputs_vae, test_source_drugs, test_source_labels, train_target_inputs_vae, train_target_drugs, train_target_labels, val_target_inputs_vae, val_target_drugs, val_target_labels, test_target_inputs_vae, test_target_drugs, test_target_labels, train_target_data_merged, val_target_data_merged, test_target_data_merged, train_source_data_merged, val_source_data_merged, test_source_data_merged = load_datasets(sample_id=fold)

In [ ]:
train_source_data_merged

In [ ]:
train_source_data_merged[["sample_id", "drug_name", "auc"]]

In [ ]:
train_target_data_merged

In [ ]:
train_target_data_merged[genes_7776]

In [ ]:
train_target_data_merged[["sample_id", "drug_name", "recist"]]

In [ ]:
# create datasets
# Cell Lines
source_dataset_train = TensorDataset(torch.FloatTensor(train_source_data_merged[genes_7776].values), torch.FloatTensor(train_source_data_merged[[str(i) for i in range(0, 2048)]].values), torch.FloatTensor(train_source_data_merged["auc"].values))
source_dataset_val = TensorDataset(torch.FloatTensor(val_source_data_merged[genes_7776].values), torch.FloatTensor(val_source_data_merged[[str(i) for i in range(0, 2048)]].values), torch.FloatTensor(val_source_data_merged["auc"].values))
source_dataset_test = TensorDataset(torch.FloatTensor(test_source_data_merged[genes_7776].values), torch.FloatTensor(test_source_data_merged[[str(i) for i in range(0, 2048)]].values), torch.FloatTensor(test_source_data_merged["auc"].values))

# Patients
target_dataset_train = TensorDataset(torch.FloatTensor(train_target_data_merged[genes_7776].values), torch.FloatTensor(train_target_data_merged[[str(i) for i in range(0, 2048)]].values), torch.FloatTensor(train_target_data_merged["recist"].values))
target_dataset_val = TensorDataset(torch.FloatTensor(val_target_data_merged[genes_7776].values), torch.FloatTensor(val_target_data_merged[[str(i) for i in range(0, 2048)]].values), torch.FloatTensor(val_target_data_merged["recist"].values))
target_dataset_test = TensorDataset(torch.FloatTensor(test_target_data_merged[genes_7776].values), torch.FloatTensor(test_target_data_merged[[str(i) for i in range(0, 2048)]].values), torch.FloatTensor(test_target_data_merged["recist"].values))


In [ ]:
# data loaders
source_dataloader_train = DataLoader(source_dataset_train, batch_size = 512, shuffle = True, worker_init_fn = seed_worker, generator = g)
target_dataloader_train = DataLoader(target_dataset_train, batch_size = 512, shuffle = True, worker_init_fn = seed_worker, generator = g)

source_dataloader_val = DataLoader(source_dataset_val, batch_size = 512, shuffle = False, worker_init_fn = seed_worker, generator = g)
target_dataloader_val = DataLoader(target_dataset_val, batch_size = 512, shuffle = False, worker_init_fn = seed_worker, generator = g)

source_dataloader_test = DataLoader(source_dataset_test, batch_size = 512, shuffle = False, worker_init_fn = seed_worker, generator = g)
target_dataloader_test = DataLoader(target_dataset_test, batch_size = 512, shuffle = False, worker_init_fn = seed_worker, generator = g)



In [ ]:
class STL(nn.Module):
    def __init__(self, cl_vae, patient_vae):
        super().__init__()
        # self.cl_vae = cl_vae
        self.patient_vae = patient_vae
        self.drug_embedder = nn.Sequential(nn.Linear(2048, 256), nn.ReLU(), nn.Linear(256, 64))
        # self.audrc_predictor = nn.Sequential(nn.Linear(64 * 2, 16), nn.ReLU(), nn.Linear(16, 1))
        self.recist_predictor = nn.Sequential(nn.Linear(64 * 2, 16), nn.ReLU(), nn.Linear(16, 1))

    def forward(self, cl_inp, cl_drug, patient_inp, patient_drug, audrc, recist):
        # cl_inp and patient_inp are 797 dim, both drugs are 2048 dim
        # cl_inp_emb, _, _, _ = self.cl_vae(cl_inp) # From VAE encoder + reparameterization
        patient_inp_emb, _, _, _ = self.patient_vae(patient_inp)

        # cl_drug_emb = self.drug_embedder(cl_drug)
        patient_drug_emb = self.drug_embedder(patient_drug)

        # cl_cat = torch.cat((cl_inp_emb, cl_drug_emb), axis = 1)
        patient_cat = torch.cat((patient_inp_emb, patient_drug_emb), axis = 1)

        # recist and audrc prediction
        # audrc_pred = self.audrc_predictor(cl_cat)
        recist_pred = self.recist_predictor(patient_cat)

        return patient_cat, recist_pred
        

In [ ]:
# train STL
mtl_model = STL(cl_vae, patient_vae).to(device)

In [ ]:
optimizer = torch.optim.Adam(params = mtl_model.parameters(), lr=1e-5)

In [ ]:
def testing_loop(model, cl_val_loader, patient_val_loader):
    model.eval()
    prediction = []
    true = []
    for idx, batch in enumerate(patient_val_loader):
        patient_inp = batch[0].to(device)
        drug_inp = batch[1].to(device)
        label = batch[2].to(device)
        with torch.no_grad():
            patient_emb, _, _, _ = model.patient_vae(patient_inp)
            drug_emb = model.drug_embedder(drug_inp)
            patient_cat = torch.cat((patient_emb, drug_emb), axis = 1)
            pred = model.recist_predictor(patient_cat)
            prediction.append(pred)
            true.append(label)
    predictions = torch.cat(prediction).view(-1, 1)
    trues = torch.cat(true).view(-1, 1)
    return nn.BCEWithLogitsLoss()(predictions, trues)
            

In [20]:
# training loop
count = 0
patient_val_losses = []
for epoch in range(300):
    mtl_model.train()
    epoch_loss = []
    for idx0, batch0 in enumerate(source_dataloader_train):
        for idx1, batch1 in enumerate(target_dataloader_train):
            optimizer.zero_grad()
            cl_inp, cl_drug, patient_inp, patient_drug, audrc, recist = batch0[0].to(device), batch0[1].to(device), batch1[0].to(device), batch1[1].to(device), batch0[2].to(device), batch1[2].to(device)
            patient_cat, recist_pred = mtl_model(cl_inp, cl_drug, patient_inp, patient_drug, audrc, recist)

            # # align both
            # coral_loss = coral(cl_cat, patient_cat)

            # losses
            # audrc_loss = nn.MSELoss()(audrc_pred.view(-1, 1), audrc.view(-1, 1))
            recist_loss = nn.BCEWithLogitsLoss()(recist_pred.view(-1, 1), recist.view(-1, 1))

            # total_loss = coral_loss + audrc_loss + recist_loss
            # total_loss = audrc_loss + recist_loss
            total_loss = recist_loss
            total_loss.backward()
            optimizer.step()

            epoch_loss.append(total_loss.cpu().detach().numpy().item())

    # get val loss
    patient_val_loss = testing_loop(mtl_model, source_dataloader_val, target_dataloader_val)
    patient_val_losses.append(patient_val_loss.item())
    print(f"Epoch {epoch}: train loss = {np.mean(epoch_loss)}, val patient loss = {patient_val_loss.item()}")

    if len(patient_val_losses) ==  1:
        best_val_loss = patient_val_loss.item()

    print(f"Best val loss = {best_val_loss}")
    print(f"Current val loss = {patient_val_loss.item()}")

    if patient_val_loss.item() <= best_val_loss: # minimize val loss
        torch.save(mtl_model.state_dict(), f"{folder_config['model_checkpoint_folder']}MTL_model_fold{fold}.pth")
        best_val_loss = patient_val_loss
        print("Saved!")
        count = 0
    else:
        print("Increased count")
        count += 1

    if count >= 3:
        print("Converged")
        break
            
    

Epoch 0: train loss = 0.6706809545654098, val patient loss = 0.65804123878479
Best val loss = 0.65804123878479
Current val loss = 0.65804123878479
Saved!
Epoch 1: train loss = 0.5887035556088865, val patient loss = 0.5765764117240906
Best val loss = 0.65804123878479
Current val loss = 0.5765764117240906
Saved!
Epoch 2: train loss = 0.4171895946747337, val patient loss = 0.6558474898338318
Best val loss = 0.5765764117240906
Current val loss = 0.6558474898338318
Increased count
Epoch 3: train loss = 0.25568385670582455, val patient loss = 0.9221434593200684
Best val loss = 0.5765764117240906
Current val loss = 0.9221434593200684
Increased count
Epoch 4: train loss = 0.14655968515214576, val patient loss = 1.3216094970703125
Best val loss = 0.5765764117240906
Current val loss = 1.3216094970703125
Increased count
Converged


In [21]:
# run inference on cell line, drug pairs to get pseudolabels
mtl_model_trained = STL(cl_vae, patient_vae).to(device)

In [22]:
mtl_model_trained.load_state_dict(torch.load(f"{folder_config['model_checkpoint_folder']}MTL_model_fold{fold}.pth"))

<All keys matched successfully>

In [23]:
mtl_model_trained.eval()

STL(
  (patient_vae): vae(
    (mu_layer): Linear(in_features=128, out_features=64, bias=True)
    (sigma_layer): Linear(in_features=128, out_features=64, bias=True)
    (encoder): Sequential(
      (enc-0): Linear(in_features=7776, out_features=512, bias=True)
      (act-0): Tanh()
      (enc-1): Linear(in_features=512, out_features=128, bias=True)
      (act-1): ReLU()
    )
    (decoder): Sequential(
      (-dec-0): Linear(in_features=64, out_features=128, bias=True)
      (-act-0): Tanh()
      (dec-0): Linear(in_features=128, out_features=512, bias=True)
      (act-0): Tanh()
      (dec-1): Linear(in_features=512, out_features=7776, bias=True)
      (act-1): Sigmoid()
    )
  )
  (drug_embedder): Sequential(
    (0): Linear(in_features=2048, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=64, bias=True)
  )
  (recist_predictor): Sequential(
    (0): Linear(in_features=128, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_fea

In [24]:
# Load augmented cell lines and drug combos
cl_augmented_df = load_augmented_cl_dataset(model_config["sample_id"])
train_val_cell_lines = list(cl_augmented_df.index)
if model_config["experiment_id"] == "1B":
    drugs_with_fp = [model_config["experiment_settings"].split(", ")[0]] # extract out drug name
elif model_config["experiment_id"] == "1A":
    drugs_with_fp = [model_config["experiment_settings"]] # has only drug name
else: # in 2A and 2B include all available drugs with fp
    drugs_with_fp = list(drug_fp.index)
possible_cl_drug_combinations = list(itertools.product(train_val_cell_lines, drugs_with_fp))
possible_cl_drug_combinations_df = pd.DataFrame(possible_cl_drug_combinations, columns = ["sample_id", "drug_name"])

Loaded augmented CL data: (1193, 64)


In [25]:
len(possible_cl_drug_combinations_df)

571447

In [26]:
# using data loaders to prevent execessive memory usage
class CustomCellLineDataSetUnlabelled(TensorDataset):
    def __init__(self, cl_augmented_df, drug_fp, possible_combinations): # possible_combinations must only consist of samples with drug name with a fingerprint
        self.possible_combinations = possible_combinations
        self.augmented_cl_df = cl_augmented_df
        self.drug_fp = drug_fp

    def __getitem__(self, idx):
        sample_name, drug_name = self.possible_combinations[idx]
        mut_profile = self.augmented_cl_df.loc[sample_name].values
        drug_inp = self.drug_fp.loc[drug_name].values
        return torch.FloatTensor(mut_profile), torch.FloatTensor(drug_inp)

    def __len__(self):
        return len(self.possible_combinations)

In [27]:
cl_aug_train_dataset = CustomCellLineDataSetUnlabelled(cl_augmented_df, drug_fp, possible_cl_drug_combinations)
print("Number of possible cl drug combos before pseudo label based filtering: ")
print(len(cl_aug_train_dataset))
cl_aug_train_dataloader = DataLoader(cl_aug_train_dataset, batch_size=model_config["source_batch_size"], shuffle=False) # to preserve order for later subset selection


Number of possible cl drug combos before pseudo label based filtering: 
571447


In [28]:
def inference_mtl(model, cl_aug_train_dataloader):
    # forward on augmented cl data, via the patient embedder, and recist predictor
    model.eval()
    pseudo_y = []
    for idx, batch in enumerate(cl_aug_train_dataloader):
        patient_inp_emb = batch[0].to(device)
        patient_drug = batch[1].to(device)
        # print(patient_inp.shape)
        with torch.no_grad():
            # patient_inp_emb, _, _, _ = model.patient_vae(patient_inp)
        
            patient_drug_emb = model.drug_embedder(patient_drug)
        
            patient_cat = torch.cat((patient_inp_emb, patient_drug_emb), axis = 1)
        
            recist_pred = nn.Sigmoid()(model.recist_predictor(patient_cat)).view(-1, 1)
            pseudo_y.append(recist_pred)

    return torch.cat(pseudo_y)

In [29]:
pseudolabels_df = pd.DataFrame()
pseudolabels_df[["sample_id", "drug_name"]] = possible_cl_drug_combinations
pseudolabels_df

,sample_id,drug_name
0,PR-132fPs,JW-7-24-1
1,PR-132fPs,KIN001-260
2,PR-132fPs,NSC-87877
3,PR-132fPs,GNE-317
4,PR-132fPs,NAVITOCLAX
...,...,...
571442,PR-2AxAKM,SB590885
571443,PR-2AxAKM,STAUROSPORINE
571444,PR-2AxAKM,TW 37
571445,PR-2AxAKM,ULIXERTINIB


In [30]:
# get pseudo labels
pseudolabels = inference_mtl(mtl_model_trained, cl_aug_train_dataloader)

In [31]:
pseudolabels_df["pseudolabels"] = pseudolabels.cpu().detach().numpy()

In [32]:
pseudolabels_df

,sample_id,drug_name,pseudolabels
0,PR-132fPs,JW-7-24-1,0.276278
1,PR-132fPs,KIN001-260,0.325867
2,PR-132fPs,NSC-87877,0.300446
3,PR-132fPs,GNE-317,0.292867
4,PR-132fPs,NAVITOCLAX,0.221668
...,...,...,...
571442,PR-2AxAKM,SB590885,0.471385
571443,PR-2AxAKM,STAUROSPORINE,0.405897
571444,PR-2AxAKM,TW 37,0.395958
571445,PR-2AxAKM,ULIXERTINIB,0.450421


In [33]:
pseudolabels_df.describe()

,pseudolabels
count,571447.000000
mean,0.351494
std,0.080653
min,0.064604
25%,0.296515
50%,0.348547
75%,0.402838
max,0.763780


In [34]:
def convert_binary(prediction, lower_threshold, upper_threshold):
    if prediction >= upper_threshold:
        return 1
    elif prediction < lower_threshold:
        return 0
    else:
        return -1

In [35]:
# threshold and select confident samples
if fold in [0, 1]:
    pseudolabels_df["pseudolabels_binary"] = pseudolabels_df["pseudolabels"].apply(lambda x: convert_binary(x, 0.1, 0.7))
else:
    pseudolabels_df["pseudolabels_binary"] = pseudolabels_df["pseudolabels"].apply(lambda x: convert_binary(x, 0.1, 0.7))

In [36]:
pseudolabels_df[pseudolabels_df.pseudolabels_binary != -1]["pseudolabels_binary"].value_counts()

pseudolabels_binary
0    121
1     67
Name: count, dtype: int64

In [37]:
# using data loaders to prevent execessive memory usage
class CustomCombinedDataSetLabelled(TensorDataset):
    def __init__(self, combined_df, cl_augmented_df, train_target_inputs_vae, drug_fp): # possible_combinations must only consist of samples with drug name with a fingerprint
        self.sample_df = combined_df.reset_index(drop=True)
        self.augmented_cl_df = cl_augmented_df
        self.tcga_vae_df = train_target_inputs_vae[~train_target_inputs_vae.index.duplicated(keep="first")]
        self.drug_fp = drug_fp

    def __getitem__(self, idx):
        row = self.sample_df.iloc[idx]
        sample_name = row["sample_id"]
        drug_name = row["drug_name"]
        if sample_name in self.tcga_vae_df.index: # using VAE version instead of mutation profiles
            mut_profile = self.tcga_vae_df.loc[sample_name].values
        if sample_name in self.augmented_cl_df.index:
            mut_profile = self.augmented_cl_df.loc[sample_name].values
        drug_inp = self.drug_fp.loc[drug_name].values
        response = row["recist"]
        return torch.FloatTensor(mut_profile), torch.FloatTensor(drug_inp), response

    def __len__(self):
        return len(self.sample_df)

In [38]:

# non-abstained, confident pseudo labels
confident_pseudolabels_df = pseudolabels_df[pseudolabels_df.pseudolabels_binary != -1]
confident_pseudolabels_df_idx = confident_pseudolabels_df.index # used to filter out the possible drug combinations df

confident_cl_drug_combinations_df = possible_cl_drug_combinations_df[possible_cl_drug_combinations_df.index.isin(confident_pseudolabels_df_idx)].copy()
confident_cl_drug_combinations_df["recist"] = list(confident_pseudolabels_df["pseudolabels_binary"])
print("Number of confident cl drug combinations with pseudolabels: ")
print(confident_cl_drug_combinations_df.shape)
print("Pseudo label distribution after majority vote:")
print(confident_cl_drug_combinations_df.recist.value_counts())

# combine confident CL samples with pseudolabels, with TCGA train data
combined_dataset_df = pd.concat([confident_cl_drug_combinations_df, train_target_data_merged[confident_cl_drug_combinations_df.columns]], axis=0)
combined_dataset = CustomCombinedDataSetLabelled(combined_dataset_df, cl_augmented_df, train_target_inputs_vae, drug_fp)
combined_dataloader = DataLoader(combined_dataset, batch_size=model_config["drp_batch_size"], shuffle=True, worker_init_fn = seed_worker, generator = g)



Number of confident cl drug combinations with pseudolabels: 
(188, 3)
Pseudo label distribution after majority vote:
recist
0    121
1     67
Name: count, dtype: int64


In [39]:
class DRP(nn.Module):
    def __init__(self):
        super().__init__()
        self.drug_embedder = nn.Sequential(nn.Linear(2048, 256), nn.ReLU(), nn.Linear(256, 64))
        self.recist_predictor = nn.Sequential(nn.Linear(64 * 2, 16), nn.ReLU(), nn.Linear(16, 1))

    def forward(self, patient_inp, patient_drug):
        # patient_inp is 64 dim, drugs are 2048 dim
        patient_drug_emb = self.drug_embedder(patient_drug)
        patient_cat = torch.cat((patient_inp, patient_drug_emb), axis = 1)

        # recist prediction
        recist_pred = self.recist_predictor(patient_cat)

        return recist_pred
        

In [40]:
def inference_drp_model(model, patient_val_dataloader):
    model.eval()
    y_preds = []
    y_trues = []
    for idx, batch in enumerate(patient_val_dataloader):
        with torch.no_grad():
            patient_inp = batch[0].to(device)
            patient_drug = batch[1].to(device)
            label = batch[2].to(device)
            y_preds.append(nn.Sigmoid()(model(patient_inp, patient_drug)).view(-1, 1))
            y_trues.append(label.view(-1, 1))
    return torch.cat(y_preds), torch.cat(y_trues)

In [41]:
def train_drp_model(model, train_dataloader, patient_val_dataloader, num_epochs=100, lr=1e-3):
    """
    To train vanilla baseline model
    """
    criterion = nn.BCEWithLogitsLoss()
    optim = torch.optim.Adam(model.parameters(), lr = lr)
    # training 
    val_corrs = []
    count = 0
    for i in range(num_epochs):
        model.train()
        train_losses = []
        if i > 10 and i % 10 == 0:
            lr = lr/10
            optim = torch.optim.Adam(model.parameters(), lr = lr)
        for idx, batch in enumerate(train_dataloader):
            optim.zero_grad()
            patient_inp = batch[0].to(device)
            patient_drug = batch[1].to(device)
            label = batch[2].to(device)
            y_pred = model(patient_inp, patient_drug).view(-1, 1)
            loss = criterion(y_pred, label.view(-1, 1).to(device, dtype=torch.float32))
            loss.backward()
            optim.step()
            train_losses.append(loss.item())

        y_test_pred, test_y = inference_drp_model(model, patient_val_dataloader)
        patient_corr = pearsonr(test_y.detach().cpu().numpy().reshape(-1), y_test_pred.detach().cpu().numpy().reshape(-1)).statistic + 1 # range in [0, 2]

        val_corrs.append(patient_corr)
        print(f"Epoch {i}: Training loss: {np.mean(train_losses)} |  Validation correlation: {patient_corr}")

        # wandb.log({
        #     f"{model.model_name}_train_loss": loss.detach().item(),
        #     f"validation_score": patient_corr
        # })
        # convergence based on val score
        if len(val_corrs) == 1: # first epoch
            best_val_score = patient_corr

        # save model
        if model_config["model_save_criteria"] in ["val_AUROC", "val_AUPRC", "val_corr"]: # maximise values
            if patient_corr >= best_val_score:
                best_val_score = patient_corr
                # save model
                print("Best model")
                torch.save(model.state_dict(), f"{folder_config['model_checkpoint_folder']}/{model.model_name}_{model_config['model_save_criteria']}_{model_config['experiment_id']}_{model_config['experiment_settings']}_fold{model_config['sample_id']}.pth")
                count = 0 # reset count
            else:
                count += 1 # declining performance on validation data
        else:
            print("Unsupported metric for optimising")
            return
        
        if count >= 3:
            print("Converged")
            break

        # # convergence checking based on validation correlation
        # if len(val_corrs) > 2:
        #     if val_corrs[-1] < val_corrs[-2]: # maximise correlation
        #         count += 1
        #     else:
        #         print("Best model")
        #         torch.save(model.state_dict(), f"{folder_config['model_checkpoint_folder']}/{model.model_name}_{model_config['model_save_criteria']}_{model_config['experiment_id']}_{model_config['experiment_settings']}_fold{model_config['sample_id']}.pth")
        #         count = 0
        # if len(val_corrs) == 1:
        #     torch.save(model.state_dict(), f"{folder_config['model_checkpoint_folder']}/{model.model_name}_{model_config['model_save_criteria']}_{model_config['experiment_id']}_{model_config['experiment_settings']}_fold{model_config['sample_id']}.pth")
        # if count > 3:
        #     print("Converged")
        #     break

In [42]:
target_val_vae_dataset = CustomCombinedDataSetLabelled(val_target_data_merged, cl_augmented_df, val_target_inputs_vae, drug_fp)
target_dataloader_val_vae = DataLoader(target_val_vae_dataset, batch_size=model_config["drp_batch_size"], shuffle=True, worker_init_fn = seed_worker, generator = g)

In [43]:
len(target_val_vae_dataset)

56

In [44]:
# initialise the DRP NN 
nn_drp = DRP().to(device)
nn_drp.model_name = "DRP_model"

# Train DRP model
train_drp_model(nn_drp, combined_dataloader, target_dataloader_val_vae, num_epochs=model_config["drp_epochs"], lr=1e-4)

Epoch 0: Training loss: 0.6850144863128662 |  Validation correlation: 0.9942123732224866
Best model
Epoch 1: Training loss: 0.6738283932209015 |  Validation correlation: 0.9966617424074469
Best model
Epoch 2: Training loss: 0.6770106852054596 |  Validation correlation: 0.9989669083725039
Best model
Epoch 3: Training loss: 0.6697863340377808 |  Validation correlation: 1.0016466806320599
Best model
Epoch 4: Training loss: 0.6703705489635468 |  Validation correlation: 1.0044690591174343
Best model
Epoch 5: Training loss: 0.6645727455615997 |  Validation correlation: 1.007424507562135
Best model
Epoch 6: Training loss: 0.6648680865764618 |  Validation correlation: 1.0108358263005952
Best model
Epoch 7: Training loss: 0.6617021560668945 |  Validation correlation: 1.0144791873637415
Best model
Epoch 8: Training loss: 0.6616109311580658 |  Validation correlation: 1.0183098920534022
Best model
Epoch 9: Training loss: 0.6572049558162689 |  Validation correlation: 1.022834186414201
Best model
Ep

In [45]:
nn_drp_trained = DRP().to(device)
nn_drp_trained.model_name = "DRP_model"
nn_drp_trained.load_state_dict(torch.load(f"{folder_config['model_checkpoint_folder']}/{nn_drp_trained.model_name}_{model_config['model_save_criteria']}_{model_config['experiment_id']}_{model_config['experiment_settings']}_fold{model_config['sample_id']}.pth"))

<All keys matched successfully>

In [46]:
nn_drp_trained.eval()

DRP(
  (drug_embedder): Sequential(
    (0): Linear(in_features=2048, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=64, bias=True)
  )
  (recist_predictor): Sequential(
    (0): Linear(in_features=128, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [47]:
train_target_inputs_vae

,vae_feat0,vae_feat1,vae_feat2,vae_feat3,vae_feat4,vae_feat5,vae_feat6,vae_feat7,vae_feat8,vae_feat9,...,vae_feat54,vae_feat55,vae_feat56,vae_feat57,vae_feat58,vae_feat59,vae_feat60,vae_feat61,vae_feat62,vae_feat63
sample_id,,,,,,,,,,,,,,,,,,,,,
P-0021780-T01-IM6,0.753870,2.671893,-0.065950,-0.772220,0.109030,-0.369968,-3.230829,4.487302,-2.270967,0.570039,...,1.957493,-1.881829,-3.369706,2.474177,3.400534,0.229281,-0.453274,-0.959082,-0.787572,2.721531
TCGA-A5-A1OH,-3.121249,0.396491,-1.609849,2.203196,0.017517,2.055745,0.465779,1.823159,0.577330,-0.769108,...,-1.059080,-1.022598,3.021566,3.092883,2.320927,-2.494244,0.989937,-2.556913,-4.543158,-1.736792
TCGA-DX-A7EQ,-1.464140,-1.723419,3.162946,-0.389088,0.841693,-1.140654,1.063845,-3.504539,-1.551661,0.125888,...,-1.422489,-6.378718,-2.549912,-0.597865,-1.969940,-1.956433,1.194958,2.477910,0.117304,0.970536
TCGA-FB-A5VM,-1.489462,-2.549048,1.081577,-0.366478,2.826008,3.115010,0.557466,-0.623550,3.350868,-4.735223,...,1.134958,-0.607584,-0.990438,0.835826,-1.021624,-0.112169,0.075469,-0.297340,-2.167166,2.701391
s_DS_bkm_035_T,-1.819688,-0.556152,1.155614,0.839682,0.148615,-2.107968,-0.875630,2.340693,-4.234872,-1.195544,...,4.688541,0.788113,1.890439,-1.989851,2.756450,2.734972,0.252763,2.669675,0.342463,-0.071207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-GN-A8LK,0.636500,-2.450130,0.376215,-1.003094,1.821947,-2.638656,0.970507,-0.832626,0.848191,-2.101937,...,0.115783,-1.103842,3.154271,1.583811,1.738071,-0.124071,-2.681928,-0.047339,0.919442,0.729093
TCGA-EX-A3L1,-2.921046,-0.991886,1.129316,0.579603,0.016393,-0.955165,1.972070,2.286034,-0.743556,-0.991338,...,1.029625,1.662122,-4.011302,2.389884,-1.478411,2.402270,1.379301,1.346545,1.760828,-2.319649
TCGA-3A-A9IC,-1.340715,2.228920,1.564497,0.221986,2.836560,1.148562,1.103388,3.092237,1.349141,2.581805,...,-0.760562,6.891579,0.049777,-1.765402,2.736322,1.370849,-1.050812,-2.966963,-1.720458,3.103694


In [48]:
# target test
target_test_vae_dataset = CustomCombinedDataSetLabelled(test_target_data_merged, cl_augmented_df, test_target_inputs_vae, drug_fp)
target_dataloader_test_vae = DataLoader(target_test_vae_dataset, batch_size=model_config["drp_batch_size"], shuffle=False)

In [49]:
len(target_test_vae_dataset)

113

In [50]:
y_test_pred, test_y = inference_drp_model(nn_drp_trained, target_dataloader_test_vae)

In [51]:
res_df = pd.DataFrame()
res_df["y_pred"] = y_test_pred.cpu().detach().numpy().reshape(-1)
res_df["y_true"] = test_y.cpu().detach().numpy().reshape(-1)

In [52]:
res_df

,y_pred,y_true
0,0.386800,1
1,0.485448,0
2,0.459732,0
3,0.417941,0
4,0.441921,0
...,...,...
108,0.503851,0
109,0.482720,1
110,0.450225,0
111,0.355868,0


In [53]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [54]:
roc_auc_score(res_df["y_true"], res_df["y_pred"])

0.5440051020408163

In [55]:
average_precision_score(res_df["y_true"], res_df["y_pred"])

0.529498208070906

In [56]:
res_df.to_csv(f"{folder_config['model_checkpoint_folder']}/prediction_patients_val_corr_2A_ALL_fold{fold}.csv")